In [ ]:
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [ ]:
import os
os.environ["LD_LIBRARY_PATH"]

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from collections import defaultdict
import json
import tensorflow as tf
import keras

In [ ]:
W=10

In [ ]:
import Clustering

In [ ]:
import Forecasting

In [ ]:
from tensorflow.python.client import device_lib 
device_lib.list_local_devices() 

In [ ]:
# import importlib
# import Clustering, Forecasting
# importlib.reload(Clustering)
# importlib.reload(Forecasting)

In [ ]:
# import tensorflow.compat.v1 as tf

In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
data = pd.read_csv("DataSet2.csv", sep=";")#, parse_dates=['Timestamp']) #, nrows=10)

In [ ]:
columns = data.columns

In [ ]:
columns
#CV, DV

In [ ]:
metafile = open("DataSet2_fix.json")
metafile = json.load(metafile)

In [ ]:
CV_DV = [x[0] for x in metafile['ColumnKind'].items() if x[1] == "CV" or x[1] == "DV"]
CV_DV = [x for x in list(columns) if x in CV_DV]
data = data[CV_DV]
CV_DV

In [ ]:
data.shape

In [ ]:
K = 30
df = data.groupby(data.index // K).mean() #усреднение
df_np = df.to_numpy()

In [ ]:
from sktime.performance_metrics.forecasting import MeanAbsoluteScaledError, MeanAbsolutePercentageError
from sklearn.metrics import mean_absolute_error as mae
mase = MeanAbsoluteScaledError(multioutput='raw_values')
mase_uni = MeanAbsoluteScaledError(multioutput='uniform_average')
mape = MeanAbsolutePercentageError(multioutput='raw_values')
# mae = MeanAbsoluteError()


#if ‘raw_values’, returns a full set of errors in case of multioutput input. If ‘uniform_average’, errors of all outputs are averaged with uniform weight.

In [ ]:
dataset = df_np[:, :]
# dataset = np.concatenate((dataset[:, :8], dataset[:, 9:]), axis=1)
dataset.shape

**__Отбор признаков на минималках__**

In [ ]:
#удаление константных
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(0.001)
dataset1 = selector.fit_transform(dataset) 
dataset1.shape

In [ ]:
set(range(dataset1.shape[-1])) - set(selector.get_feature_names_out(input_features=np.arange(dataset.shape[-1])))

In [ ]:
print(np.unique(dataset[:, 14]))

In [ ]:
# plt.plot(dataset[:, 8], color="green")
plt.plot(np.concatenate((dataset[:32714, 8], dataset[32717:, 8])))
plt.title("consumption_09")
plt.xlabel("time")
plt.ylabel("value")
plt.show()

In [ ]:
# values = np.unique(dataset[:, 8])
# print(values)
# print([len(np.argwhere(dataset[:, 8] == val)) for val in values])
# print(f"Изменение только в {np.argwhere(dataset[1:, 8] - dataset[:-1, 8])}")

**Эти датчики пока выкинем из-за константности: ** 

8: consumption_09

14 : consumption_07 


In [ ]:
columns[15], columns[9]

In [ ]:
plt.plot(dataset[7])    

In [ ]:
# from scipy.signal import correlate
# a = np.array([np.sin(x + 1) for x in range(10)])
# b = np.array([np.sin(x + 2) for x in range(10)])
# c = correlate(a, b, mode="full") #mode='same'
# plt.plot(a, label="a")
# plt.plot(b, label="b")
# plt.plot(c, label="correlation")
# plt.legend()
# c.shape

In [ ]:
dataset = dataset1
dataset.shape

**Прогнозирование**

In [ ]:
dataset = dataset1[:]
dataset.shape

In [ ]:
plt.plot(dataset[..., 8])

In [ ]:
from sklearn.model_selection import train_test_split
window_sizes_for_clustering = 10
# X, y = dataset[:-window_sizes_for_clustering, ...], dataset[window_sizes_for_clustering:, ...]
# X_train, y_train, X_test, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
n_split = round(0.2 * dataset.shape[0])
dataset_train, dataset_test = dataset[:-n_split, ...], dataset[-n_split:, ...]

In [ ]:
dataset_train.shape, dataset_test.shape

In [ ]:
window_sizes_for_clustering = [1, 5, 10]#[1, 5, 10, 15] #[1, 3, 5, 10, 15]
Ns_clusters = [2, 9, 11] #, 3 5, 7]#[2, 5, 7, 9, 11, 13]

In [ ]:
clusters_algorithms = ["MeanShift", "Kmeans"] #["Agglomerative"] #["MeanShift"]
clustering_algorithms = []
clustering_algorithms += [Clustering.Kmeans_for_windows(W=W, N_clusters=N_cluster) for W in window_sizes_for_clustering for N_cluster in Ns_clusters]
# clustering_algorithms += [Clustering.MeanShift_for_windows(W=W) for W in window_sizes_for_clustering]
clustering_algorithms += [Clustering.AgglomerativeClustering_for_windows(W=W, N_clusters=N_cluster) for W in window_sizes_for_clustering for N_cluster in Ns_clusters]

In [ ]:
clustering_algorithms[0].N_clusters

In [ ]:
# Ns_clusters = [2]
# window_sizes_for_clustering = [7]

In [ ]:
# N, M, Q = 100, 100, 2
# dataset_train = np.column_stack([[np.sin(x / 40) for x in range(N)], [0.01 + np.sin(x / 20) for x in range(N)]])
# # dataset_train = np.array([np.sin(x / 10000) for x in range(N)])[:, None]
# # dataset_train = np.column_stack(([np.sqrt(x / 10) for x in range(N)], [x + ])
# dataset_test = np.column_stack([[np.sin(x / 30) for x in range(M)], [0.01 + np.sin(x / 30)*1.01 for x in range(M)]])

In [ ]:
dataset_train.shape

In [ ]:
# %%time
# a = Clustering.MeanShift_for_windows(W=W).fit_predict(dataset_train)

In [ ]:
#clusters_labels, clusters_model 
# clustering_results = [Clustering.apply_clustering(dataset, cur_cluster_alg) for cur_cluster_alg in clustering_algorithms]
# clusters_labels = [x[0] for x in clustering_results]
# clusters_models = [x[1] for x in clustering_results]
clustering_results = [cur_cluster_alg.fit_predict(dataset_train) for cur_cluster_alg in clustering_algorithms]
clusters_labels = clustering_results

In [ ]:
plt.hist(clustering_results[0])

In [ ]:
W=20

In [ ]:
# models, results_training 
# forecasting_training_results = [Forecasting.apply_forecasting_training(dataset, cur_cluster_labels, W=10) for cur_cluster_labels in clusters_labels]
# models = [x[0] for x in forecasting_training_results]
# results_training = [x[1] for x in forecasting_training_results]
# metrics = [x[2] for x in forecasting_training_results]
#_simple_version
forecasting_training_results = [Forecasting.apply_forecasting_training_simple_version(dataset_train, cur_cluster_labels, W=W) for cur_cluster_labels in clusters_labels]
models = [x[0] for x in forecasting_training_results]
results_training = [x[1] for x in forecasting_training_results]
metrics_training = [x[2] for x in forecasting_training_results]

In [ ]:
# results_training = results_training[0]

In [ ]:
assert(len(models) == len(results_training))

In [ ]:
Q = dataset.shape[-1]

In [ ]:
len(results_training)

In [ ]:
plt.plot(results_training[0][:100, 8], label="true")
plt.plot(results_training[0][:100, Q+8], label="predicted")
plt.legend()
plt.title("Оригинальные значения")
plt.show()

In [ ]:
# import csv
# # full_results = np.concatenate((results_training, results_testing), axis=0)
# # full_results = results_training
# full_results = np.concatenate([results_training[:, :2*Q], results_training[:, 4*Q:]], axis=1)
# with open("output_table_03-24.csv", "w") as fout:
#     writer = csv.writer(fout)
#     writer.writerow(["real "+str(i) for i in range(Q)] + ["predicted " + str(i) for i in range(Q)] + ["cluster_num", "mode"])
#     for i in range(full_results.shape[0]):
#         writer.writerow(full_results[i])


In [ ]:
# print("weighted mae:", [x[-1] for x in forecasting_training_results])
print(np.mean([x['mae'] for x in metrics_training[2]], axis=1))
print(np.mean([x['mase'] for x in metrics_training[2]], axis=1))
# plt.plot([x['mae'] for x in metrics_training[0]][0])

In [ ]:
clusters_sizes = [[np.sum(cur_clusters_labels == i) for i in range(len(np.unique(cur_clusters_labels)))] for cur_clusters_labels in clusters_labels]
N_clusters = len(clusters_sizes)
mae_on_max_cluster = np.array([metrics_training[i][np.argmax(clusters_sizes[i])]['mae'] for i in range(len(forecasting_training_results))])


In [ ]:
# maes = np.array([[metrics_training[i][c]['mae'] for c in range(N_clusters)] for i in range(len(metrics_training))])
# # weighted_mae = np.array([np.average(maes[i, :, :], weights=clusters_sizes[i], axis=0) for i in range(len(forecasting_training_results))])


In [ ]:
np.mean(mae_on_max_cluster, axis=1)

In [ ]:
len(models[0])

In [ ]:
results_testing = [Forecasting.predict_through_clusters(dataset_test, clustering_algorithms[i], models[i], W=W) for i in range(len(results_training))]

In [ ]:
len(results_testing)

In [ ]:
[str(x) for x in clustering_algorithms]

Выбор алгоритма

In [ ]:
full_results = [np.concatenate((
                    results_training[i],
                    np.concatenate((dataset_test[W:], results_testing[i]), axis=1)
                    ),
                    axis=0
                ) for i in range(len(results_training))]

In [ ]:
#tmp
# full_results = results_testing
# full_results = [np.concatenate((dataset_test[W:], results_testing[i]), axis=1) for i in range(len(results_training))]
# full_results = results_training

In [ ]:
global_metrics = [Forecasting.calc_metrics_for_full_results(x) for x in full_results]

In [ ]:
# global_metrics
clusters_sizes = [np.array([np.sum(x[:, -2] == i) for i in range(len(np.unique(x[:, -2])))]) for x in full_results]
N_clusters = [len(x) for x in clusters_sizes]
print(N_clusters)
# mae_on_max_cluster = np.array([global_metrics[i][np.argmax(clusters_sizes[i])]['mae'] for i in range(len(full_results))])
# mae_mean = np.array([np.mean(global_metrics[i][np.argmax(clusters_sizes[i])]['mae']) for i in range(full_results)])

In [ ]:
maes = np.array([global_metrics[i]['mae'] for i in range(len(forecasting_training_results))])
mae_on_max_cluster = np.array([
        Forecasting.calc_metrics_for_full_results(
            full_results[i][full_results[i][:, -2] == np.argmax(clusters_sizes[i])]
        )['mae'] 
    for i in range(len(full_results))])

In [ ]:
np.around(maes, decimals=3)

In [ ]:
print(np.around(np.mean(mae_on_max_cluster, axis=1).reshape(2, 3, 3),decimals=3))

In [ ]:
plt.plot(np.mean(mae_on_max_cluster, axis=1), label="on max cluster")
plt.plot(np.mean(maes, axis=1), label="global mae")
plt.legend()
plt.title("Average mae")
plt.ylabel("mae")
plt.xlabel("Number of experiment")
plt.show()

In [ ]:
ind1 = np.argmin(np.mean(mae_on_max_cluster, axis=1))
ind1

In [ ]:
plt.hist(clusters_labels[ind1])
plt.title("Размеры кластеров")

In [ ]:
plt.hist(full_results[0][full_results[0][:, -1] == 3, 5])
plt.show()
plt.hist(full_results[0][full_results[0][:, -1] == 0, 5])

In [ ]:
np.mean(global_metrics[ind1]['mase'])

In [ ]:
plt.plot([np.mean(x['mase']) for x in global_metrics])

In [ ]:
ind2 = np.argmin([np.mean(x['mase']) for x in global_metrics])
ind2, np.mean(global_metrics[ind2]['mase'])

In [ ]:
num = (np.abs(full_results[15][:, :Q] - full_results[15][:, Q:2*Q])).sum(axis=0) / full_results[15].shape[0]
denom = (np.abs(full_results[15][1:, :Q] - full_results[15][:-1, :Q])).sum(axis=0) / (full_results[15].shape[0] - 1)

In [ ]:
denom.mean(), num.mean()

In [ ]:
np.mean(num / denom)

Тестирование (старое)

In [ ]:
#номер сохраняемого эксперимента
IND = 0

In [ ]:
ws = np.array([1, 1, 1, 5, 5, 5, 10, 10, 10] + [1, 1, 1, 5, 5, 5, 10, 10, 10])
len(ws)

In [ ]:
ns = N_clusters
ns

In [ ]:
algs = ["Kmeans"] * 9 + ["Agglomer"] * 9

In [ ]:
IND = 3

In [ ]:
import csv
# tmp = np.concatenate((dataset_test[W:], results_testing[0]), axis=1)
# full_results = np.zeros((results_training.shape[0] + results_testing.shape[0], results_training))
# full_results = np.concatenate((results_training[0], tmp), axis=0)
# full_results = results_training
# full_results = np.concatenate([results_training[:, :2*Q], results_training[:, 4*Q:]], axis=1)
for IND in range(0, 18):
    with open(f"./results/W20_group30/{IND:02}_{algs[IND]}_N{N_clusters[IND]}_w{ws[IND]}.csv", "w") as fout:
        writer = csv.writer(fout)
        writer.writerow(["real "+str(i) for i in range(Q)] + ["predicted " + str(i) for i in range(Q)] + ["cluster_num", "mode"])
        for i in range(full_results[IND].shape[0]):
            writer.writerow(full_results[IND][i])


In [ ]:
data.columns

In [ ]:
read = pd.read_csv("./output_table_04-07_for60_2.csv")
new_data = pd.DataFrame(read, columns=columns)

In [ ]:
np.mean(mae_on_max_cluster, axis=1)

In [ ]:
# plt.plot(np.mean([x[-1] for x in forecasting_training_results], axis=1), label="weighted MAE")
# plt.plot(np.mean(mae_on_max_cluster, axis=1), label="MAE on max cluster")
# plt.legend()
# plt.title("MAE on different cluster algorithms")
# plt.show()

In [ ]:
clustering_results1 = [Clustering.apply_clustering(dataset, cur_cluster_alg) for cur_cluster_alg in [Clustering.MeanShift_for_windows(W=W) for W in window_sizes_for_clustering]]


In [ ]:
clusters_labels1 = [x[0] for x in clustering_results1]

clusters_sizes1 = [[np.sum(cur_clusters_labels == i) for i in range(len(np.unique(cur_clusters_labels)))] for cur_clusters_labels in clusters_labels]


In [ ]:
plt.hist(clusters_labels1[-1])
plt.yscale("log")
plt.show()

In [ ]:
np.argmin(np.mean(mae_on_max_cluster, axis=1))

In [ ]:
parameters = {"dif":True, 
              "cluster_algs":clustering_algorithms}
# models, model_mae, results_training = Forecasting.try_parameters(parameters, dataset_train)

In [ ]:
model_mae

In [ ]:
a = np.random.random((5, 4))
sc = Forecasting.MyStandardScaler()
sc.fit(a)
sc.transform(a)
sc.mean, a

Сохранение моделей

In [ ]:
IND = 14

In [ ]:
# import os
# for i, file_name in enumerate(list(os.listdir("scalers"))):
#     os.unlink("scalers/"+file_name)
# for i in range(len(models['scalers'])):
#     with open("scalers/"+str(i)+".pkl", "wb") as f:
#         pickle.dump(models['scalers'][i], f)

In [ ]:
os.unlink("clusters_model.pkl")
with open("clusters_model.pkl", "wb") as f:
#     pickle.dump(models['clusters_model'], f)
        pickle.dump(clustering_algorithms[IND], f)


In [ ]:
import os, shutil

for i, file_name in enumerate(list(os.listdir("models"))):
    shutil.rmtree("models/"+file_name)
for i in range(len(models[IND])):
    if not isinstance(models[IND][i], int):
        models[IND][i].save("models/"+str(i))
print(len(models))

Чтение моделей

In [ ]:
import os
N_clusters = len(os.listdir('models'))
assert(len(os.listdir('scalers')) == N_clusters)
models = {'models':[], 'clusters_model':[], 'scalers':[]}

In [ ]:
from tensorflow import keras
for file_name in os.listdir('models'):
    models['models'].append(keras.models.load_model('models/'+file_name))

In [ ]:
for file_name in os.listdir('scalers'):
    with open('scalers/'+file_name, 'rb') as f:
        models['scalers'].append(pickle.load(f))

In [ ]:
with open('clusters_model.pkl', 'rb') as f:
    models['clusters_model'] = pickle.load(f)

Тестирование

In [ ]:
clusters_model = models["clusters_model"]
forecasting_models = models['models']
scalers = models['scalers']
assert(len(forecasting_models) == len(scalers))

In [ ]:
import importlib
import Clustering, Forecasting
importlib.reload(Clustering)
importlib.reload(Forecasting)

In [ ]:
window_sizes_for_clustering

In [ ]:
N_clusters = 7

In [ ]:
# dataset_windows = sliding_window_view(dataset_train, (window_size_clustering, dataset_train.shape[-1])) #(N, 1, W, Q)
# new_shape = dataset_windows.shape
# dataset_windows = dataset_windows.reshape(new_shape[0], new_shape[2] * new_shape[3])
# cluster_nums = clusters_model.labels_
# print(f"{dataset.shape=}, {dataset_windows.shape=}, {cluster_nums.shape=}, {dataset.shape[0] - dataset_windows.shape[0]}")
# cluster_nums = np.pad(cluster_nums, (dataset.shape[0] - dataset_windows.shape[0], 0), mode='constant', constant_values=(cluster_nums[0])) #-1
# print(f"After pad: {dataset.shape=}, {cluster_nums.shape=}")
# # if window_size_clustering > window_size_forecasting:
# #     cluster_num = np.pad(cluster_nums, (0, ), mode='constant', constant_values=(cluster_nums[-1]))
# y_pred = np.zeros((dataset.shape[0] - window_size_forecasting, dataset.shape[-1]))
# #only if dif then +1
# dataset_windows = sliding_window_view(dataset, (window_size_forecasting + 1, dataset.shape[-1]))
# cluster_nums = cluster_nums[window_size_forecasting:] #+1
    
# full_results = np.zeros((cluster_nums.shape[0], 2 * dataset_train.shape[-1] + 2)) # real_Q, Q, cluster_num, mask

# for cluster_num in range(N_clusters):
#     mask = (cluster_nums == cluster_num)
#     if np.sum(mask) == 0:
#         continue

#     cur_windows = dataset_windows[mask, 0, ...] #(M, Wf, Q)
#     scalers[cluster_num].save_first_elements(cur_windows)
#     cur_windows = np.array(scalers[cluster_num].transform(cur_windows))
#     cur_pred = np.array(prediction_models[cluster_num](cur_windows)) #(M, Q)
#     cur_pred = scalers[cluster_num].inverse_transform(cur_pred)
#     cur_pred = scalers[cluster_num].add_first_elements(cur_pred)
#     y_pred[mask] = cur_pred
#     Q = dataset.shape[-1]
#     full_results[mask, Q:2 * Q] = cur_pred
#     full_results[mask, 2 * Q] = cluster_num
#     full_results[mask, 2 * Q + 1] = 3 #global test

In [ ]:
# window_size_for_clustering = clusters_model.cluster_centers_.shape[-1] // dataset_test.shape[-1]
window_size_for_clustering = window_sizes_for_clustering[0]
y_pred, results_testing = Forecasting.predict_through_clusters(dataset_test, clusters_model, forecasting_models, scalers, window_size_clustering=window_size_for_clustering)

In [ ]:
#лучшие параметры
window_size_for_clustering, len(forecasting_models)

In [ ]:
y_pred.shape, dataset.shape[-1]

In [ ]:
print(f"{y_pred.shape=}, {dataset_test.shape=}")
y_true = dataset_test[-y_pred.shape[0]:]
print(y_true.shape, results_testing[:, :dataset_train.shape[-1]].shape)
results_testing[:, :dataset_train.shape[-1]] = y_true
cur_mase = Forecasting.my_mase(y_true, y_pred, multioutput='raw_values')
cur_mae = Forecasting.my_mae(y_true, y_pred, multioutput="raw_values")

In [ ]:
Q = dataset_train.shape[-1]
Q

In [ ]:
import csv
# full_results = np.concatenate((results_training, results_testing), axis=0)
full_results = results_training
with open("output_table_03-23.csv", "w") as fout:
    writer = csv.writer(fout)
    writer.writerow(["real "+str(i) for i in range(Q)] + ["predicted " + str(i) for i in range(Q)] + ["cluster_num", "mode"])
    for i in range(full_results.shape[0]):
        writer.writerow(full_results[i])


In [ ]:
# cur_mase, cur_mae

In [ ]:
# full_results.shape

In [ ]:
mask = cur_mase <= np.percentile(cur_mase, 60)
plt.plot(np.arange(cur_mase.shape[0])[mask], cur_mase[mask])
plt.yscale("log")
plt.xlabel("features")
plt.ylabel("MASE")
plt.title("MASE")
plt.show()

In [ ]:
mask = cur_mae <= np.percentile(cur_mae, 60)
# plt.plot(np.arange(cur_mae.shape[0])[mask], cur_mae[mask])
plt.plot(cur_mae[mask])
plt.xlabel("features")
plt.ylabel("MAE")
plt.title("best MAE")
# plt.show()
plt.savefig("best MAE.png")

In [ ]:
plt.plot(cur_mase[cur_mase <= np.percentile(cur_mase, 100)])
plt.yscale("log")
plt.xlabel("features")
plt.ylabel("MASE")
plt.title("50% best MASE")
plt.show()

In [ ]:
cur_mape = mape(y_true, y_pred)
plt.plot(cur_mape[cur_mape < np.percentile(cur_mape, 50)])
plt.yscale("log")
plt.xlabel("features")
plt.ylabel("MAPE")
plt.title("50% best MAPE")
plt.show()

**Мелкое тестирование**

In [ ]:
N, Q = 1000, 10
# test_array = (np.random.random((N * Q)) + np.sin(np.arange(N * Q) / 100)).reshape(N, Q)
test_array = (np.sin(np.arange(N*Q)/100) * 5).reshape(N, Q)

In [ ]:
test_array

In [ ]:
W = 5
sc = Forecasting.MyStandardScaler()
clusters_X, clusters_labels = Clustering.split_to_clusters(test_array, np.zeros(test_array.shape[0]), W=W + 2)
test_array_windows = clusters_X[0]
X_true, y_true = Forecasting.split_X_y(test_array_windows)
prepared_array = sc.fit_transform(test_array_windows)
prepared_X, prepared_y = Forecasting.split_X_y(prepared_array)
y_changed = sc.inverse_transform(prepared_y)
# array_changed = sc.inverse_transform(prepared_array)

In [ ]:
plt.plot(y_true[:100, 0], label="true")
plt.plot(prepared_y[:100, 0], label="transformed")
plt.legend()
plt.show()

In [ ]:
np.max(np.abs(y_true - y_changed))

In [ ]:
test_labels = np.zeros(N)
# test_labels[test_array[:, 0] <= np.percentile(test_array, 50)] = 1

In [ ]:
test_array.shape

In [ ]:
models, test_results, _ = Forecasting.apply_forecasting_training_simple_version(test_array, test_labels)

In [ ]:
Q = test_array.shape[-1]
plt.plot(test_results[:40, 0], label="true")
plt.plot(test_results[:40, Q], label="predicted")
plt.legend()
plt.show()

In [ ]:
import csv
with open("output_table_03-22.csv", "w") as fout:
    writer = csv.writer(fout)
    writer.writerow(["real "+str(i) for i in range(Q)] + ["predicted " + str(i) for i in range(Q)] + ["cluster_num", "mode"])
    for i in range(test_results.shape[0]):
        writer.writerow(test_results[i])
